# Tuning Script

In [4]:
# Parameters subject to change in every script - involved in naming

MODEL = 'mlp'
TUNER = 'jiaochengb'

---

In [5]:
# detect whether this is a google environment

COLAB_ENVIRONMENT = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENVIRONMENT = True
except:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import sys
import os
import pickle

if COLAB_ENVIRONMENT:
    home_directory = './drive/MyDrive/LAB/AFLBrownlowPredictor2024/' # my home directory is stored in ./LAB of google drive
    if MODEL == 'ebr':
        !pip install interpret==0.5.0
    elif MODEL == 'cbr':
        !pip install catboost
    elif MODEL in ['tf', 'mlp']:
        !pip install PyTorch2Sklearn==0.0.5
    !pip install JXAutoML==0.2.8
else:
    home_directory = '../../'

from JXAutoML.JiaoChengB import JiaoChengB as Tuner

import pandas as pd

In [7]:
# Parameters that are same for every script
SEED = 42

TASK_TYPE = 'Regression'  # this task is a classification task

# we have decided to tune features as hp
TUNE_FEATURES_AS_HYPERPARAMETERS = True

PyTorchModel = True

---

In [8]:
# model to use to tune
from PyTorch2Sklearn.MLP import MLP as clf
from PyTorch2Sklearn.environment import *
from PyTorch2Sklearn.utils.data import TabularDataFactory, TabularDataset

# what values to try for each hyperparameter
parameter_choices = {
    'hidden_dim': (4, 16, 32, 64, 128, 256, 512),
    'hidden_layers': (2, 3, 4, 5),
    'dropout': (0, 0.05, 0.1, 0.15, 0.2),
    'batch_size': (4, 8, 16, 32, 64, 128, 256, 512, 1024),
    'epochs': (5, 10, 15, 20),
    'lr': (1e-5, 1e-4, 1e-3, 1e-2, 1e-1),
    'batchnorm': (False, True),
    'grad_clip': (False, True),
}

# what values to set non-tuneable parameters/hyperparameters
non_tunable_hyperparameters_dict = {
    'random_state': SEED,
    'loss': nn.MSELoss(),
    'rootpath': './',
    'TabularDataFactory': TabularDataFactory,
    'TabularDataset': TabularDataset,
    'name': 'MLP',
    'output_dim': 1,
    'mode': 'Regression',
}

# what order to tune hyperparameters in (JiaoCheng specific)
tuning_order = ['features',
                'hidden_layers',
                'hidden_dim',
                'dropout',
                'batchnorm',
                'grad_clip',
                'lr',
                'epochs',
                'batch_size'
                ]

# what order to tune hyperparameters in (JiaoCheng specific)
default_hyperparameter_values = {
    'features': 0,
    'hidden_layers': 2,
    'hidden_dim': 16,
    'dropout': 0.1,
    'batchnorm': False,
    'grad_clip': True,
    'lr': 1e-2,
    'epochs': 10,
    'batch_size': 32
}

---

Read in and Prepare Data

In [9]:
train_data = pd.read_csv(f'{home_directory}data/curated/modelling/train.csv')
val_data = pd.read_csv(f'{home_directory}data/curated/modelling/val.csv')
test_data = pd.read_csv(f'{home_directory}data/curated/modelling/test.csv')

train_X = train_data.drop(columns=['target'])
train_y = train_data['target']
val_X = val_data.drop(columns=['target'])
val_y = val_data['target']
test_X = test_data.drop(columns=['target'])
test_y = test_data['target']

In [10]:
# Import feature importance ordering

with open(f'{home_directory}models/feature_importance_ordering.pickle', 'rb') as f:
    feature_importance_ordering = pickle.load(f)

Set parameters

In [11]:
# initialisation
tuner = Tuner()

print('---')

# define what model we are tuning
tuner.read_in_model(
    clf, TASK_TYPE, pytorch_model=PyTorchModel, optimised_metric='r2')

print('---')

# read in the data for training and validation
tuner.read_in_data(train_X, train_y, val_X, val_y, test_X, test_y)

print('---')

# set what hp values to tune
tuner.set_hyperparameters(parameter_choices)
# WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up hp values that need to be changed from default but NOT to be tuned
tuner.set_non_tuneable_hyperparameters(non_tunable_hyperparameters_dict)

print('---')

# set up feature importance ordering

if TUNE_FEATURES_AS_HYPERPARAMETERS:
    tuner.set_features(feature_importance_ordering)
    # WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up the order of hyperparameters when iteratively tuning using JiaoCheng
tuner.set_tuning_order(tuning_order)

print('---')

# set up the default hp values for first iteration of tuning JiaoCheng
tuner.set_hyperparameter_default_values(default_hyperparameter_values)

print('---')

try:  # try to read in previous results to continue tuning ...
    tuner.read_in_tuning_result_df(
        f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')
except:
    print('No previous tuning data read in')

print('---')

# set up where to save the tuning result csv
tuner.set_tuning_result_saving_address(
    f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')

print('---')

# set up where to save the current best model
tuner.set_best_model_saving_address(
    f'{home_directory}models/tmp_models/{TUNER}_{MODEL}')

JiaoCheng Initialised
---
Successfully read in model <class 'PyTorch2Sklearn.MLP.MLP'>, which is a Regression model optimising for r2
---
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
---
Successfully recorded hyperparameter choices
---
Successfully recorded non_tuneable_hyperparameter choices
---
Successfully recorded tuneable feature combination choices and updated relevant internal structures
---
---
---
Error message: 8.0
Error Importing this Row: (41, hidden_dim                                                                     8.0
hidden_layers                                                                  2.0
dropout                                                                        0.1
batch_size                                                                    32.0
epochs                                                                        10.0
lr                                                

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
# begin tuning ... (or continue training)

tuner.tune()

Streaming output truncated to the last 5000 lines.

Best combo after this hyperparameter: (5, 0, 0, 3, 2, 1, 0, 0, 36) , NOT UPDATED SINCE LAST HYPERPARAMETER


Round 3 
Hyperparameter: lr (index: 5) 

Already Trained and Tested combination (5, 0, 0, 3, 2, 0, 0, 0, 36), which had val score of 0.9013
        Current best combo: (5, 0, 0, 3, 2, 1, 0, 0, 36) with val score 0.9052. 
        Has trained 281 of 3729600 combinations so far
Already Trained and Tested combination (5, 0, 0, 3, 2, 1, 0, 0, 36), which had val score of 0.9052
        Current best combo: (5, 0, 0, 3, 2, 1, 0, 0, 36) with val score 0.9052. 
        Has trained 281 of 3729600 combinations so far
Already Trained and Tested combination (5, 0, 0, 3, 2, 2, 0, 0, 36), which had val score of 0.9045
        Current best combo: (5, 0, 0, 3, 2, 1, 0, 0, 36) with val score 0.9052. 
        Has trained 281 of 3729600 combinations so far
Already Trained and Tested combination (5, 0, 0, 3, 2, 3, 0, 0, 36), which had val score of 0